# Notes

In [1]:
from typing import List, Optional
from pydantic import BaseModel, Field
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.output_parsers import PydanticOutputParser
# -------------------------
# 1) Pydantic schema for Notes
# -------------------------
class Note(BaseModel):
    topic: str = Field(description="Main topic")
    summary: str = Field(description="Short summary in simple language")
    key_points: List[str] = Field(description="Bullet points explaining the main idea")
    timestamp: float = Field(description="Timestamp where this note is relevant")

class NotesOutput(BaseModel):
    notes: List[Note] = Field(description="List of summarized notes with key points")

notes_parser = PydanticOutputParser(pydantic_object=NotesOutput)
notes_format_instructions = notes_parser.get_format_instructions()


# -------------------------
# 2) System prompt for Notes
# -------------------------
notes_system_message = SystemMessagePromptTemplate.from_template(
    """You are an expert note-taker.

You will receive a YouTube transcript with timestamps.

Your task:
1. Create clear study NOTES for the video.
2. Each note must have:
   - Topic
   - Summary (2–3 sentences max, simple English)
   - Key bullet points
   - Timestamp (approx in seconds)
3. Be concise but complete. Make notes helpful for revision.
4. Do not invent content not in transcript.
5. Output must be valid JSON and match this schema:
{format_instructions}
"""
)

# -------------------------
# 3) Human prompt (with transcript)
# -------------------------
notes_human_message = HumanMessagePromptTemplate.from_template(
    """Transcript:
{transcript}

Now generate study notes."""
)

notes_chat_prompt = ChatPromptTemplate.from_messages(
    [notes_system_message, notes_human_message]
)
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

# -------------------------
# 4) Runner function for Notes
# -------------------------
def generate_notes_from_transcript(transcript: str) -> NotesOutput:
    prompt = notes_chat_prompt.format_prompt(
        transcript=transcript, 
        format_instructions=notes_format_instructions
    )
    messages = prompt.to_messages()

    response_message = model.invoke(messages)
    raw_output = response_message.content

    return notes_parser.parse(raw_output)


In [2]:
from youtube_transcript_api import YouTubeTranscriptApi
import re
def load_transcript(url: str) -> str | None:
    """
    Fetch transcript for a YouTube video.
    """
    pattern = r'(?:v=|\/)([0-9A-Za-z_-]{11})'
    match = re.search(pattern, url)
    if match:
        video_id = match.group(1)
        try:
            captions = YouTubeTranscriptApi().fetch(video_id,languages=['en','hi']).snippets
            data = [f"{item.text} ({item.start})" for item in captions]
            return " ".join(data)
        except Exception as e:
            print(f"❌ Error fetching transcript: {e}")
            return None

captions = load_transcript("https://youtu.be/nQa31xdXbGk")
notes = generate_notes_from_transcript(captions)

In [18]:

for note in notes.notes:
    print(f"Topic: {note.topic}")
    print(f"Summary: {note.summary}")
    print("-" * 80)

Topic: Introduction to MCP Architecture
Summary: This video is a continuation of the previous one, focusing on the 'What' aspect of MCP. It will detail the Multi-Agent Communication Protocol (MCP) architecture, building from a basic version to a complete diagram.
--------------------------------------------------------------------------------
Topic: Simplest MCP Architecture: Host and Server
Summary: The most basic MCP architecture involves two entities: a Host and a Server. The Host is an AI chatbot that interacts with the user, while the Server is a tool capable of executing specific tasks.
--------------------------------------------------------------------------------
Topic: MCP Architecture: Introducing the Client
Summary: In a more refined MCP architecture, the Host does not directly communicate with the Server. Instead, it uses an intermediary called the MCP Client, which acts as a helper to facilitate communication.
--------------------------------------------------------------